In [1]:
import torch

from utils import masking
import ppo
from dataloader import SeqComb
train_set = SeqComb.get_SeqComv('onetwo', 'TRAIN')
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

/home/hjkim/anaconda3/envs/torchrl/lib/python3.8/site-packages/sklearn/utils/multiclass.py:13: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix


In [2]:
from models import Predictor

d_in = 1
d_model = 128
d_out = 3
seq_len = 100
device = 'cuda:4'
backbone = 'rnn'

predictor = Predictor.PredictorNetwork(d_in=d_in, d_model=d_model, d_out=d_out, seq_len=seq_len, backbone=backbone)
predictor = predictor.to(device)
pred_optim = torch.optim.Adam(predictor.parameters(), lr=1e-4)

mask_fn = masking.MaskingFunction('seq')

ppo.predictor_random_train(
    pre_train_epochs=100,
    loader=train_loader,
    predictor=predictor,
    pred_optim=pred_optim,
    seq_len=seq_len,
    mask_fn=mask_fn,
    device=device,
)

/home/hjkim/anaconda3/envs/torchrl/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 100/100 [07:05<00:00,  4.25s/it, loss=0.717]


In [20]:
backbone = 'tcn'

predictor = Predictor.PredictorNetwork(d_in=d_in, d_model=d_model, d_out=d_out, seq_len=seq_len, backbone=backbone)
predictor = predictor.to(device)
pred_optim = torch.optim.Adam(predictor.parameters(), lr=1e-4)

mask_fn = masking.MaskingFunction('seq')

ppo.predictor_random_train(
    pre_train_epochs=100,
    loader=train_loader,
    predictor=predictor,
    pred_optim=pred_optim,
    seq_len=seq_len,
    mask_fn=mask_fn,
    device=device,
)

/home/hjkim/anaconda3/envs/torchrl/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
 52%|█████▏    | 52/100 [04:17<03:57,  4.94s/it, loss=0.704]


KeyboardInterrupt: 

In [ ]:
backbone = 'rnn'

predictor = Predictor.PredictorNetwork(d_in=d_in, d_model=d_model, d_out=d_out, seq_len=seq_len, backbone=backbone)
predictor = predictor.to(device)
pred_optim = torch.optim.Adam(predictor.parameters(), lr=1e-4)

mask_fn = masking.MaskingFunction('seq')

ppo.predictor_random_train(
    pre_train_epochs=100,
    loader=train_loader,
    predictor=predictor,
    pred_optim=pred_optim,
    seq_len=seq_len,
    mask_fn=mask_fn,
    device=device,
)